In [1]:
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objects as go
import platform, plotly
import numpy as np
from numpy import pi


#print(f'Python version: {platform.python_version()}')
#print(f'Plotly version: {plotly.__version__}')


#Exemplo data
#data = [['',      'Emma', 'Isabella', 'Ava', 'Olivia', 'Sophia', 'row-sum'],
#        ['Emma',     16, 3, 28, 0, 18, 65],
#        ['Isabella', 18, 0, 12, 5, 29, 64],
#        ['Ava',       9, 11, 17, 27, 0, 64],
#        ['Olivia',  19, 0, 31, 11, 12, 73],
#        ['Sophia',  23, 17, 10, 0, 34, 84]]

data = [['','USD','CNY','BRL','EUR','GBP','CAD','JPY'],
['USD',1,7.10485,5.70309,0.92655,0.81028,1.41489,106.111],
['CNY',0.14075,1,0.8027,0.13041,0.11405,0.19914,14.9341],
['BRL',0.17534,1.24579,1,0.16246,0.14208,0.24809,18.5888],
['EUR',1.07928,7.6681,6.15521,1,0.87452,1.52706,114.532],
['GBP',1.23414,8.76837,7.0384,1.14349,1,1.74636,130.956],
['CAD',0.70677,5.02149,4.03076,1.52706,0.57265,1,75.0455],
['JPY',0.00942,0.06696,0.0538,0.00873,0.00764,0.01333,1]]

table = ff.create_table(data, index=True)
#py.iplot(table, filename='Test.jpg')

In [2]:

matrix = np.array([[1,7.10485,5.70309,0.92655,0.81028,1.41489,106.111],
[0.14075,1,0.8027,0.13041,0.11405,0.19914,14.9341],
[0.17534,1.24579,1,0.16246,0.14208,0.24809,18.5888],
[1.07928,7.6681,6.15521,1,0.87452,1.52706,114.532],
[1.23414,8.76837,7.0384,1.14349,1,1.74636,130.956],
[0.70677,5.02149,4.03076,1.52706,0.57265,1,75.0455],
[0.00942,0.06696,0.0538,0.00873,0.00764,0.01333,1]], dtype=int)


def check_data(data_matrix):
    L, M = data_matrix.shape
    if L != M:
        raise ValueError('Data array must have a (n,n) shape')
    return L

L = check_data(matrix)

#from IPython.display import IFrame
#IFrame('https://plot.ly/~empet/12234/', width=377, height=420)

Python version: 3.7.4
Plotly version: 4.6.0


A chord diagram encodes information in two graphical objects:

ideograms, represented by distinctly colored arcs of circles;
ribbons, that are planar shapes bounded by two quadratic Bezier curves and two arcs of circle,that can degenerate to a point;

In [3]:
#definindo ideogram
def moduloAB(x, a, b): #maps a real number onto the unit circle identified with 
                       #the interval [a,b), b-a=2*PI
        if a>= b:
            raise ValueError('Incorrect interval ends')
        y = (x-a) % (b-a)
        return y+b if y < 0 else y+a
    

def test_2PI(x):
     return 0 <= x < 2*pi
    
row_sum = [matrix[k,:].sum() for k in range(L)]

#set the gap between two consecutive ideograms
gap = 2*pi*0.005
ideogram_length = 2*pi * np.asarray(row_sum) / sum(row_sum) - gap*np.ones(L)

def get_ideogram_ends(ideogram_len, gap):
    ideo_ends = []
    left = 0
    for k in range(len(ideogram_len)):
        right = left + ideogram_len[k]
        ideo_ends.append([left, right])   
        left = right + gap
    return ideo_ends

ideo_ends = get_ideogram_ends(ideogram_length, gap)

def make_ideogram_arc(R, phi, a=50):
    # R is the circle radius
    # phi is the list of  angle coordinates of an arc ends
    # a is a parameter that controls the number of points to be evaluated on an arc
    if not test_2PI(phi[0]) or not test_2PI(phi[1]):
        phi = [moduloAB(t, 0, 2*pi) for t in phi]
    length = (phi[1]-phi[0]) % 2*pi 
    nr = 5 if length <= pi/4 else int(a*length/pi)

    if phi[0] < phi[1]:   
        theta = np.linspace(phi[0], phi[1], nr)
    else:
        phi = [moduloAB(t, -pi, pi) for t in phi]
        theta = np.linspace(phi[0], phi[1], nr)
    return R * np.exp(1j*theta)

make_ideogram_arc(1.3, [11*pi/6, pi/17])

array([1.12583302-0.65j      , 1.14814501-0.60972373j,
       1.16901672-0.5686826j , 1.18842197-0.5269281j ,
       1.20633642-0.48451259j, 1.22273759-0.44148929j,
       1.23760491-0.39791217j, 1.25091973-0.3538359j ,
       1.26266534-0.30931575j, 1.27282702-0.26440759j,
       1.28139202-0.21916775j, 1.28834958-0.17365297j,
       1.29369099-0.12792036j, 1.29740954-0.08202728j,
       1.29950058-0.0360313j , 1.29996146+0.01000988j,
       1.29879163+0.0560385j , 1.29599253+0.10199682j,
       1.2915677 +0.1478272j , 1.28552267+0.19347214j,
       1.27786503+0.23887437j])

In [18]:
#definindo cores
labels=['Emma', 'Isabella', 'Ava', 'Olivia', 'Sophia']
ideo_colors=['rgba(244, 109, 67, 0.75)',
             'rgba(253, 174, 97, 0.75)',
             'rgba(254, 224, 139, 0.75)',
             'rgba(217, 239, 139, 0.75)',
             'rgba(166, 217, 106, 0.75)']

In [5]:
def map_data(data_matrix, row_value, ideogram_length):
    mapped = np.zeros(data_matrix.shape)
    for j  in range(L):
        mapped[:, j] = ideogram_length * data_matrix[:,j] / row_value
    return mapped



mapped_data = map_data(matrix, row_sum, ideogram_length)
mapped_data

array([[0.27949818, 0.05240591, 0.48912181, 0.        , 0.31443545],
       [0.31429952, 0.        , 0.20953301, 0.08730542, 0.50637144],
       [0.15714976, 0.19207193, 0.29683843, 0.47144927, 0.        ],
       [0.33291045, 0.        , 0.54316969, 0.19273763, 0.21025923],
       [0.40429305, 0.2988253 , 0.17577959, 0.        , 0.5976506 ]])

In [6]:
#IFrame('https://plot.ly/~empet/12519/', width=420, height=420)

In [7]:
#The array idx_sort, defined below, has on each row the indices that sort the corresponding row in mapped_data:
idx_sort = np.argsort(mapped_data, axis=1)
idx_sort

array([[3, 1, 0, 4, 2],
       [1, 3, 2, 0, 4],
       [4, 0, 1, 2, 3],
       [1, 3, 4, 0, 2],
       [3, 2, 1, 0, 4]], dtype=int64)

In [8]:

def make_ribbon_ends(mapped_data, ideo_ends,  idx_sort):
    L = mapped_data.shape[0]
    ribbon_boundary = np.zeros((L,L+1))
    for k in range(L):
        start = ideo_ends[k][0]
        ribbon_boundary[k][0] = start
        for j in range(1,L+1):
            J = idx_sort[k][j-1]
            ribbon_boundary[k][j] = start + mapped_data[k][J]
            start = ribbon_boundary[k][j]
    return [[(ribbon_boundary[k][j], ribbon_boundary[k][j+1] ) for j in range(L)] for k in range(L)]




In [9]:

def control_pts(angle, radius):
    #angle is a  3-list containing angular coordinates of the control points b0, b1, b2
    #radius is the distance from b1 to the  origin O(0,0) 

    if len(angle) != 3:
        raise InvalidInputError('angle must have len =3')
    b_cplx = np.array([np.exp(1j*angle[k]) for k in range(3)])
    b_cplx[1] = radius * b_cplx[1]
    return list(zip(b_cplx.real, b_cplx.imag))



def ctrl_rib_chords(l, r, radius):
    # this function returns a 2-list containing control poligons of the two quadratic Bezier
    #curves that are opposite sides in a ribbon
    #l (r) the list of angular variables of the ribbon arc ends defining 
    #the ribbon starting (ending) arc 
    # radius is a common parameter for both control polygons
    if len(l)!=2 or len(r)!=2:
        raise ValueError('the arc ends must be elements in a list of len 2')
    return [control_pts([l[j], (l[j]+r[j])/2, r[j]], radius) for j in range(2)]

In [10]:
ribbon_color = [L * [ideo_colors[k]] for k in range(L)]

In [11]:
ribbon_color[0][4]=ideo_colors[4]
ribbon_color[1][2]=ideo_colors[2]
ribbon_color[2][3]=ideo_colors[3]
ribbon_color[2][4]=ideo_colors[4]


def make_q_bezier(b):# defines the Plotly SVG path for a quadratic Bezier curve defined by the 
                     #list of its control points
    if len(b) != 3:
        raise valueError('control poligon must have 3 points')
    A, B, C = b    
    return f'M {A[0]}, {A[1]} Q {B[0]}, {B[1]} {C[0]}, {C[1]}'



b=[(1,4), (-0.5, 2.35), (3.745, 1.47)]

#make_q_bezier(b)

'M 1, 4 Q -0.5, 2.35 3.745, 1.47'

In [12]:
#make_ribbon_arc returns the Plotly SVG path corresponding to an arc represented by its end angular coordinates theta0, theta1.

def make_ribbon_arc(theta0, theta1):

    if test_2PI(theta0) and test_2PI(theta1):
        if theta0 < theta1:
            theta0 = moduloAB(theta0, -pi, pi)
            theta1 = moduloAB(theta1, -pi, pi)
            if theta0  *theta1 > 0:
                raise ValueError('incorrect angle coordinates for ribbon')
    
        nr = int(40 * (theta0 - theta1) / pi)
        if nr <= 2: nr = 3
        theta = np.linspace(theta0, theta1, nr)
        pts=np.exp(1j*theta)# points in polar complex form, on the given arc
    
        string_arc = ''
        for k in range(len(theta)):
            string_arc += f'L {pts.real[k]}, {pts.imag[k]} '
        return   string_arc 
    else:
        raise ValueError('the angle coordinates for an arc side of a ribbon must be in [0, 2*pi]')


#make_ribbon_arc(np.pi/3, np.pi/6)

'L 0.5000000000000001, 0.8660254037844386 L 0.5877852522924732, 0.8090169943749473 L 0.6691306063588582, 0.7431448254773941 L 0.7431448254773944, 0.6691306063588581 L 0.8090169943749475, 0.5877852522924731 L 0.8660254037844387, 0.49999999999999994 '

In [13]:
#Finally we are ready to define data and layout for the Plotly plot of the chord diagram.
def plot_layout(title, plot_size):

    return dict(title=title,
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                showlegend=False,
                width=plot_size,
                height=plot_size,
                margin=dict(t=25, b=25, l=25, r=25),
                hovermode='closest',
                 )


In [14]:
#Function that returns the Plotly shape of an ideogram:
def make_ideo_shape(path, line_color, fill_color):
    #line_color is the color of the shape boundary
    #fill_collor is the color assigned to an ideogram
    
    return  dict(line=dict(color=line_color, 
                           width=0.45),
                 path=path,
                 layer='below',
                 type='path',
                 fillcolor=fill_color)

In [15]:
#We generate two types of ribbons: a ribbon connecting subarcs in two distinct ideograms, respectively a ribbon 
#from one ideogram to itself (it corresponds to mapped_data[k][k], i.e. it gives the flow of comments from a community member to herself).

def make_ribbon(l, r, line_color, fill_color, radius=0.2):
    #l=[l[0], l[1]], r=[r[0], r[1]]  represent the opposite arcs in the ribbon 
    #line_color is the color of the shape boundary
    #fill_color is the fill color for the ribbon shape
    
    poligon = ctrl_rib_chords(l,r, radius)
    b, c = poligon  
           
    return  dict(line=dict(color=line_color, 
                             width=0.5),
                 path=make_q_bezier(b) + make_ribbon_arc(r[0], r[1])+
                         make_q_bezier(c[::-1]) + make_ribbon_arc(l[1], l[0]),
                 type='path',
                 layer='below',
                 fillcolor = fill_color,    
        )

def make_self_rel(l, line_color, fill_color, radius):
    #radius is the radius of Bezier control point b_1
    
    b = control_pts([l[0], (l[0]+l[1])/2, l[1]], radius) 
    
    return  dict(line = dict(color=line_color, 
                             width=0.5),
                 path =  make_q_bezier(b)+make_ribbon_arc(l[1], l[0]),
                 type = 'path',
                 layer = 'below',
                 fillcolor = fill_color   
                )
def invPerm(perm):
    # function that returns the inverse of a permutation, perm
    inv = [0] * len(perm)
    for i, s in enumerate(perm):
        inv[s] = i
    return inv


layout=plot_layout('Chord diagram', 400)

radii_sribb = [0.4, 0.30, 0.35, 0.39, 0.12]# these value are set after a few trials

In [20]:
ribbon_info = []
shapes = []
for k in range(L):
    
    sigma = idx_sort[k]
    sigma_inv = invPerm(sigma)
    ribbon_ends = make_ribbon_ends(mapped_data, ideo_ends, idx_sort)
    for j in range(k, L):
        if matrix[k][j] == 0 and matrix[j][k]==0: continue
        eta = idx_sort[j]
        eta_inv = invPerm(eta)
        l = ribbon_ends[k][sigma_inv[j]]  
        
        if j == k:
            shapes.append(make_self_rel(l, 'rgb(175,175,175)' ,
                                    ideo_colors[k], radius=radii_sribb[k])) 
            z = 0.9*np.exp(1j*(l[0]+l[1])/2)
            
            #the text below will be displayed when hovering the mouse over the ribbon
            text = f'{labels[k]} commented on {int(matrix[k][k])} of herself Fb posts'

            ribbon_info.append(go.Scatter(x=[z.real],
                                       y=[z.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=ideo_colors[k]),
                                       text=text,
                                       hoverinfo='text'
                                       )
                              )
        else:
            r = ribbon_ends[j][eta_inv[k]]
            zi = 0.9 * np.exp(1j*(l[0]+l[1])/2)
            zf = 0.9 * np.exp(1j*(r[0]+r[1])/2)
            #texti and textf are the strings that will be displayed when hovering the mouse 
            #over the two ribbon ends
            texti = f'{labels[k]} commented on {int(matrix[k][j])} of {labels[j]} Fb posts'
            textf = f'{labels[j]} commented on {int(matrix[j][k])} of {labels[k]} Fb posts'
            
            ribbon_info.append(go.Scatter(x=[zi.real],
                                          y=[zi.imag],
                                          mode='markers',
                                          marker=dict(size=0.5, color=ribbon_color[k][j]),
                                          text=texti,
                                          hoverinfo='text'
                                       )
                              ),
            ribbon_info.append(go.Scatter(x=[zf.real],
                                          y=[zf.imag],
                                          mode='markers',
                                          marker=dict(size=0.5, color=ribbon_color[k][j]),
                                          text=textf,
                                          hoverinfo='text'
                                       )
                              )
            r = (r[1], r[0]) # IMPORTANT!!!  Reverse these arc ends because otherwise you get
                          # a twisted ribbon
            #append the ribbon shape
            shapes.append(make_ribbon(l, r, 'rgb(175,175,175)' , ribbon_color[k][j]))

In [21]:
ideograms = []
for k in range(len(ideo_ends)):
    z =  make_ideogram_arc(1.1, ideo_ends[k])
    zi = make_ideogram_arc(1.0, ideo_ends[k])
    m = len(z)
    n = len(zi)
    ideograms.append(go.Scatter(x=z.real,
                                y=z.imag,
                                mode='lines',
                                line=dict(color=ideo_colors[k], shape='spline', width=0.25),
                                text=f'{labels[k]} <br>{int(row_sum[k])} comments', 
                                hoverinfo='text'
                             )
                     )
    
       
    path = 'M '
    for s in range(m):
        path += f'{z.real[s]}, {z.imag[s]} L '
        
    Zi = np.array(zi.tolist()[::-1]) 

    for s in range(m):
        path += f'{Zi.real[s]}, {Zi.imag[s]} L '
    path += f'{z.real[0]} ,{z.imag[0]}' 
   
    shapes.append(make_ideo_shape(path,'rgb(150,150,150)' , ideo_colors[k]))
    
    

data = ideograms + ribbon_info
layout['shapes'] = shapes
fig = go.Figure(data=data, layout=layout)
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
init_notebook_mode(connected=True)
iplot(fig)